# Import modules

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import seaborn as sns
import sys
sys.path.insert(0, os.getcwd()+"/../../../libs/common")
from FileUtil import gpfs2WinPath

# Import the caldata in gpfs, & feature scaling

In [57]:
#caldatafile = r'/gpfs/WW/BD/MXP/SHARED/SEM_IMAGE/IMEC/Case02_calaveras_v3/3Tmp/CT_KPI_test/Calaveras_v3_regular_CT_KPI_003_slope_modified_revert_all_patterns/h/cache/dummydb/result/MXP/job1/ContourSelectModelCalibration452result1/caldata.txt'
caldatafile = r'/gpfs/WW/BD/MXP/SHARED/SEM_IMAGE/IMEC/Case02_calaveras_v3/3Tmp/ContourSelection/020_AEI_contour_selection_training/h/cache/dummydb/result/MXP/job1/ContourSelectModelCalibration430result1/caldata.txt'
caldatafile = gpfs2WinPath(caldatafile)

df = pd.read_csv(caldatafile, sep='\s+')

tgtColName = 'UserLabel'
neighborColNames = ['NeighborOrientation', 'NeighborParalism']#, 'NeighborParalism', 'NeighborContinuity',
allColNames = ['NeighborContinuity', 'NeighborOrientation', 'NeighborParalism', 'slope', 'intensity', 'ridge_intensity', 'contrast', 'EigenRatio']
srcColNames = ['slope', 'intensity', 'ridge_intensity', 'NeighborOrientation', 'NeighborParalism']
# srcColNames = df.columns.values
tgtColName = 'UserLabel'
outColName = 'ClfLabel'

In [58]:
wiScaling = True
scaling = lambda X_Arr: np.array([(X_Arr[i] - Xmin[i])/(Xmax[i] - Xmin[i]) for i in range(len(allColNames)) ])
if wiScaling:
    X_cal = df.loc[df.usage=='CAL', allColNames].values
    Xmin = X_cal.min(axis=0)
    Xmax = X_cal.max(axis=0)
    Xminmax = pd.DataFrame(data= np.array([Xmin, Xmax]), columns=allColNames, index=['min', 'max'])
    print(Xminmax)
    df.loc[:, allColNames] = df.loc[:, allColNames].apply(scaling, axis=1)

X_cal = df.loc[df.usage=='CAL', allColNames]
y_cal = df.loc[df.usage=='CAL', tgtColName]
X_ver = df.loc[df.usage=='VER', allColNames]
y_ver = df.loc[df.usage=='VER', tgtColName]

     NeighborContinuity  NeighborOrientation  NeighborParalism     slope  \
min            0.722482             0.766492          0.560799 -0.013827   
max            1.642146             1.000000          1.000000  0.035596   

     intensity  ridge_intensity  contrast  EigenRatio  
min   0.199917         0.002565  0.000000   -0.999942  
max   0.969997         0.047745  0.223779    0.171573  


In [ ]:
df.loc[:, tgtColName].value_counts()

In [15]:
y_cal.value_counts()

1    27221
0     6321
Name: UserLabel, dtype: int64

In [10]:
X_cal.loc[:, allColNames].describe()

,NeighborContinuity,NeighborOrientation,NeighborParalism,slope,intensity,ridge_intensity,contrast,EigenRatio
count,33542.000000,33542.000000,33542.000000,33542.000000,33542.000000,33542.000000,33542.000000,33542.000000
mean,0.387994,0.988373,0.995262,0.303452,0.528526,0.389276,0.233010,0.737328
std,0.156404,0.028994,0.020471,0.020705,0.166873,0.141172,0.127040,0.244171
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.289212,0.988118,0.996603,0.291335,0.449009,0.290756,0.138931,0.649267
50%,0.306697,0.995475,0.998885,0.303444,0.553353,0.385919,0.215535,0.817687
75%,0.464822,0.998533,0.999665,0.316025,0.633751,0.481554,0.309789,0.911365
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
df.loc[:, allColNames].describe()

,NeighborContinuity,NeighborOrientation,NeighborParalism,slope,intensity,ridge_intensity,contrast,EigenRatio
count,37525.000000,37525.000000,37525.000000,37525.000000,37525.000000,37525.000000,37525.000000,37525.000000
mean,0.393308,0.988195,0.995115,0.303479,0.524291,0.383471,0.230306,0.735731
std,0.160404,0.029189,0.020580,0.020337,0.169350,0.140480,0.126013,0.244781
min,-0.011476,0.000000,0.000000,0.000000,-0.003850,-0.003994,0.000000,0.000000
25%,0.289242,0.987932,0.996508,0.291625,0.445913,0.285821,0.136448,0.646508
50%,0.307305,0.995425,0.998861,0.303337,0.546780,0.380778,0.212922,0.815857
75%,0.484089,0.998526,0.999659,0.315830,0.629527,0.475247,0.306049,0.911118
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.loc[:, srcColNames].head(2)

In [ ]:
df.loc[:, srcColNames].describe()

# feature selection

In [ ]:
%matplotlib auto

# hist plot 

df2 = pd.melt(df, id_vars=tgtColName, value_vars=neighborColNames, value_name='value')
bins=np.linspace(df2.value.min(), df2.value.max(), 100)
g = sns.FacetGrid(df2, col="variable", hue=tgtColName, palette="Set1", col_wrap=3)
g.map(plt.hist, 'value', bins=bins, ec="k")
plt.yscale('log')
g.axes[-1].legend()

# scatter plot 
#sns.pointplot('NeighborOrientation', 'NeighborParalism', hue=tgtColName, data=df)

plt.show()

In [14]:
%matplotlib auto
def sephist(df, col):
    TP = df[df[tgtColName] == 0][col]
    TN = df[df[tgtColName] == 1][col]
    return TP, TN
df_cal = X_cal.copy(deep=True)
df_cal.loc[:, tgtColName] = y_cal
for num, alpha in enumerate(allColNames):
    plt.subplot(2, 4, num+1)

    TP, TN = sephist(df_cal, alpha)
    plt.hist((TP, TN), bins=25, alpha=0.5, label=map(''.join, zip(2*[tgtColName], 2*['=='], ['0', '1'])), color=['b', 'g'])
    #plt.hist(TP, bins=50, alpha=0.5, label=tgtColName+'==0', color='b')
    #plt.hist(TN, bins=50, alpha=0.5, label=tgtColName+'==1', color='g')
    plt.legend(loc='upper right')
    plt.title(alpha)
    plt.yscale('log')
#plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

Using matplotlib backend: Qt4Agg


In [ ]:
df.loc[:, tgtColName].value_counts()

In [ ]:
df.loc[df[tgtColName]==0, srcColNames].describe()

In [ ]:
df.loc[df[tgtColName]==1, srcColNames].describe()

# Model Calibration, and model selection

In [ ]:
def computeConfusionMatrixBinary(y_true, y_pred):
    ''' compute the confusion matrix in below format
        [TP      FN]
        [FP      TN]
    '''
    cm = np.zeros((2,2), dtype=int)
    for i, y0 in enemurate(y_true):
        if y0 == y_pred[0] == 0 :
            cm[0, 0] += 1   # TP
        elif y0 == 0 and y_pred[0] == 1 :
            cm[0, 1] += 1   # FN
        elif y0 == 1 and y_pred[0] == 0 :
            cm[1, 0] += 1   # FP
        elif y0 == 1 and y_pred[0] == 1 :
            cm[1, 1] += 1   # TN
    return cm

In [60]:
srcColNames = [ 'intensity', 'ridge_intensity', 'slope', 'contrast', 'NeighborParalism']

## SVM

In [46]:
modeltype = 'svm'
from sklearn import svm
clf = svm.SVC(kernel='linear') # , class_weight='balanced' {0: 10, 1: 1}
#allColNames = ['slope', 'intensity', 'ridge_intensity', 'contrast', 'EigenRatio', 'NeighborOrientation', 'NeighborParalism']
model = clf.fit(X_cal[srcColNames], y_cal)
modelform = pd.DataFrame(data=clf.coef_.flatten(), index=srcColNames)
modelform.loc['intercept', 0] = clf.intercept_
print(modelform)

                             0
intensity            23.293968
ridge_intensity      -1.860256
slope                14.791464
NeighborOrientation  -0.306555
NeighborParalism      0.161123
intercept           -12.502340


## Decision Tree

In [61]:
modeltype='tree'
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=0, max_depth=len(srcColNames)+1, min_samples_split=3)#class_weight='balanced'
model = clf.fit(X_cal[srcColNames], y_cal)
feature_importance = pd.DataFrame(data=model.feature_importances_.flatten(), index=srcColNames)
print(feature_importance)

#print(model.decision_path(X_cal))
print(model.get_params())

                         0
intensity         0.967004
ridge_intensity   0.006414
slope             0.024118
contrast          0.001637
NeighborParalism  0.000827
{'presort': False, 'splitter': 'best', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'min_weight_fraction_leaf': 0.0, 'criterion': 'gini', 'random_state': 0, 'max_features': None, 'max_depth': 6, 'class_weight': None}


In [55]:
# visualization for decision tree model
import graphviz 
dot_data = tree.export_graphviz(model, out_file=None, 
                         feature_names=srcColNames,  
                         class_names=tgtColName,  
                         filled=True, rounded=True,  
                         special_characters=True) 
graph = graphviz.Source(dot_data)  
graph

ImportError: No module named graphviz

In [62]:
# port decision tree model into c code
from sklearn_porter import Porter

porter = Porter(model, language='c')
output = porter.export(embed_data=True)
print(output)


# Save model:
with open('ContourSelBuiltInTreeModel.cc', 'w') as f:
    f.write(output)

#include <stdlib.h>
#include <stdio.h>
#include <math.h>

int predict(float features[5]) {

    int classes[2];
        
    if (features[0] <= 0.385035157204) {
        if (features[0] <= 0.374575078487) {
            if (features[2] <= 0.323027670383) {
                if (features[0] <= 0.350465655327) {
                    if (features[2] <= 0.31005769968) {
                        classes[0] = 5590; 
                        classes[1] = 0; 
                    } else {
                        if (features[2] <= 0.310163080692) {
                            classes[0] = 0; 
                            classes[1] = 1; 
                        } else {
                            classes[0] = 58; 
                            classes[1] = 0; 
                        }
                    }
                } else {
                    if (features[2] <= 0.302373826504) {
                        if (features[1] <= 0.15952372551) {
                            classes[0] = 1; 
           

In [ ]:
# build a classifier
from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import tree

clf = tree.DecisionTreeClassifier(random_state=0, max_depth=len(srcColNames)+1, min_samples_split=3) # 

X, y = df.loc[:, srcColNames].values, df.loc[:, tgtColName].values


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [len(srcColNames), len(srcColNames)+1, None],
              "max_features": sp_randint(3, len(srcColNames)-1),
              "min_samples_split": sp_randint(2, 8),
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {"max_depth": [len(srcColNames), None],
              "max_features": [3, len(srcColNames)-1],
              "min_samples_split": [2, 3, 8],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

## Random Forest

In [ ]:
modeltype='forest'
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10, random_state=0)
model = clf.fit(X_cal, y_cal)
feature_importance = pd.DataFrame(data=model.feature_importances_.flatten(), index=srcColNames)
print(feature_importance)

## Model performace

In [51]:
from sklearn.metrics import confusion_matrix
calcRMS = lambda y_pred, y: np.sqrt(np.mean(np.power(y_pred - y, 2)))
def printModelPerformance(modeltype, cm, usage='CAL', from_sklearn=False, use_total=True):
    if not use_total:
        if from_sklearn: # if cm from sklearn.metrics import confusion_matrix
            #               Truth
            # Predict:  [TP      FP] 
            #           [FN      TN]
            cm_norm = cm.astype('float') / cm.sum(axis=0).reshape((1, 2))
        else: # if cm by using ContourSelBaseModel.computeConfusionMatrix
            #               Predict
            # Truth:    [TP      FN] 
            #           [FP      TN]
            cm_norm = cm.astype('float') / cm.sum(axis=1).reshape((2, 1))
    else:
        cm_norm = cm.astype('float') / cm.sum()
    print("{} model on {} set, \nFN(missing) rate = {:.3f}%, \nFP(false alarm) rate = {:.3f}%".format(modeltype, usage, cm_norm[0, 1]*100, cm_norm[1, 0]*100))
    print("{} model confusion matrix on {} set:\n{}\n{}".format(modeltype, usage, cm, np.round(100*cm_norm, 3)))
    
def predict(X, y, usage='CAL'):
    y_pred = model.predict(X)
    cm = confusion_matrix(y, y_pred)
    return cm
printModelPerformance(modeltype, predict(X_cal[srcColNames], y_cal), usage='CAL')
printModelPerformance(modeltype, predict(X_ver[srcColNames], y_ver), usage='VER')

tree model on CAL set, 
FN(missing) rate = 0.447%, 
FP(false alarm) rate = 0.224%
tree model confusion matrix on CAL set:
[[ 6171   150]
 [   75 27146]]
[[ 18.398   0.447]
 [  0.224  80.931]]
tree model on VER set, 
FN(missing) rate = 0.025%, 
FP(false alarm) rate = 2.937%
tree model confusion matrix on VER set:
[[ 654    1]
 [ 117 3211]]
[[  1.64200000e+01   2.50000000e-02]
 [  2.93700000e+00   8.06180000e+01]]


In [ ]:
%matplotlib auto
import sys
import os.path
sys.path.insert(0, os.getcwd()+"/../../../libs/tacx")
print(os.getcwd()+"/../../../libs/tacx")
from SEMContour import *
sys.path.insert(0, os.getcwd()+"/../../../libs/common")
from FileUtil import gpfs2WinPath

import glob

CWD = ''.join(['/gpfs/WW/BD/MXP/SHARED/SEM_IMAGE/IMEC/Case02_calaveras_v3/3Tmp/CT_KPI_test/Calaveras_v3_regular_CT_KPI_003_slope_modified_revert_all_patterns/'
      'h/cache/dummydb/result/MXP/job1/ContourSelectModelCalibration430result1'])

''' # comment block 1 starts
#################
# type 1, review model apply image by random permutation
#################
pathfilter = '*_image_contour.txt'
pathex = gpfs2WinPath(os.path.join(CWD, pathfilter))
contourfiles = glob.glob(pathex)
np.random.RandomState(128)
contourindice = np.random.permutation(np.arange(len(contourfiles)))
for ii in range(0*8, 1*8):
    fig = plt.figure()
    for jj, idx in enumerate(contourindice[ii*8:(ii+1)*8]):
        contourfile = contourfiles[idx]
        patternid = os.path.basename(contourfile).strip('_image_contour.txt')
        ################# end of type 1
''' # comment block 1 ends
        
#################
# type 2, review model apply image by giving list
#################
patternids = [461, 1001]

for ii in range(int(np.ceil(len(patternids)/8.))):
    fig = plt.figure()
    for jj, idx in enumerate(range(ii*8, (ii+1)*8)):
        patternid = str(patternids[idx])
        contourfile = gpfs2WinPath(os.path.join(CWD, patternid+'_image_contour.txt'))
        ################# end of type 2        
        
        if not os.path.exists(contourfile):
            print(patternid+' not exist')
            continue

        # get contour data
        contour = SEMContour()
        if not contour.parseFile(contourfile):
            continue
        df = contour.toDf()

        X_test = df.loc[:, srcColNames].values
        X_test = np.array([(X_test[:,i] - Xmin[i])/(Xmax[i] - Xmin[i]) for i in range(len(srcColNames)) ]).T
        df.loc[:, 'ClfLabel'] = model.predict(X_test)
        # SEM Contour Selection resulst plot: by classifer Positive 0, & Negative 1
        def plotContourDiscriminator(contour, im=None, wndname=''):
            # plot image and classified contour point
            
            ax = fig.add_subplot(2,4,jj+1)

            imw, imh = contour.getshape()
            ax.set_aspect('equal')
            xini, yini, xend, yend = contour.getBBox()
            ax.set_xlim([xini, xend])
            ax.set_ylim([yini, yend])
            ax.set_title(wndname)

            df = contour.toDf()
            Positive = df.ClfLabel==0
            Negative = df.ClfLabel==1

            # calculate confusion matrix
            cm = np.array([len(df.loc[flt, :]) for flt in [Positive, Negative]])
            cm_norm = cm.astype('float') / cm.sum(axis=1).reshape((2,1))

            if im is not None:
                ax.imshow(im)
            ax.plot(df.loc[Positive ,'offsetx'], df.loc[Positive, 'offsety'], #'b.', markersize=1, 
                    linestyle='None', marker= 'o', markeredgecolor='r', markersize=2, markeredgewidth=1, markerfacecolor='none', 
                    label='remove: {}({:.3f}%)'.format(cm[0], cm_norm[0]*100 )) #Discriminator Positive, ClfLabel=0
            ax.plot(df.loc[Negative ,'offsetx'], df.loc[Negative, 'offsety'], #'r*', markersize=2,
                    linestyle='None', marker= '.', markeredgecolor='b', markersize=2, markeredgewidth=1, markerfacecolor='none', 
                    label='Keep: {}({:.3f}%)'.format(cm[1], cm_norm[1]*100 )) #Discriminator Negative, ClfLabel=1:

            #ax = plt.gca() # gca() function returns the current Axes instance
            #ax.set_ylim(ax.get_ylim()[::-1]) # reverse Y
            plt.gca().invert_yaxis()
            plt.legend(loc=1)
            plt.show()
        plotContourDiscriminator(contour.fromDf(df), wndname='Pattern '+ patternid)

In [ ]:
from __future__ import print_function

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

print(__doc__)

# Loading the Digits dataset
digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.